In [291]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataprep.eda import *
import seaborn as sns
from datetime import datetime

# Loading the Data

In [340]:
desc = pd.read_excel('data-dictionary.xlsx', index_col=0)
data_features = pd.read_csv('transactions_obf.csv')
data_labels = pd.read_csv('labels_obf.csv')
fraud_transactions = data_labels['eventId'].tolist()

# Creating Target Variable

In [338]:
data_features.loc[data_features['eventId'].isin(fraud_transactions) , 'isFraud'] = 1
data_features.loc[data_features['eventId'].isin(fraud_transactions) == False , 'isFraud'] = 0

In [339]:
counts = data_features['isFraud'].value_counts()
print(f'Out of the total {len(data_features)} transactions, {dict(counts).get(0)} are genuine and {dict(counts).get(1)} are fraud.')


Out of the total 118621 transactions, 117746 are genuine and 875 are fraud.


# Dataset Visualisation

# transaction time

In [329]:
# transaction time has a very high cardinality with 99.6% unique values. It should not be used to train the model as it does not provide much information ot the model. Instead I have extracted the hour from the time and created a new feature which signifies the part of the day when the transaction was made. This can be a great feature as the model can spot patterns to identify suspicious behaviour in unusual time of the day. 

In [330]:
data_features['transactionTime'] = pd.to_datetime(data_features['transactionTime'])

In [331]:
data_features['transactionTime'][1000].hour

10

In [332]:
# def get_pod(x):
#     for i in range (len(data_features)):
#         if (x[i].hour > 4) and (x[i].hour <= 8):
#             return 'Early Morning'
#         elif (x[i].hour > 8) and (x[i].hour <= 12 ):
#             return 'Morning'
#         elif (x[i].hour > 12) and (x[i].hour <= 16):
#             return'Noon'
#         elif (x[i].hour > 16) and (x[i].hour <= 20) :
#             return 'Eve'
#         elif (x[i].hour > 20) and (x[i].hour <= 24):
#             return'Night'
#         elif (x[i].hour <= 4):
#             return'Late Night'

In [341]:
def get_pod(x):
    if (x.hour > 4) and (x.hour <= 8):
        return 'Early Morning'
    elif (x.hour > 8) and (x.hour <= 12 ):
        return 'Morning'
    elif (x.hour > 12) and (x.hour <= 16):
        return'Noon'
    elif (x.hour > 16) and (x.hour <= 20) :
        return 'Eve'
    elif (x.hour > 20) and (x.hour <= 24):
        return'Night'
    elif (x.hour <= 4):
        return'Late Night'

In [342]:
data_features['transactionTime'] = pd.to_datetime(data_features['transactionTime'])
data_features['part_of_day'] = data_features['transactionTime'].apply(get_pod)
pod_dummies = pd.get_dummies(data_features['part_of_day'],prefix = 'tx_at')
data_features = pd.concat([data_features,pod_dummies],axis=1)
data_features = data_features.drop(columns=['transactionTime'])


In [343]:
data_features

,eventId,accountNumber,merchantId,mcc,merchantCountry,merchantZip,posEntryMode,transactionAmount,availableCash,part_of_day,tx_at_Early Morning,tx_at_Eve,tx_at_Late Night,tx_at_Morning,tx_at_Night,tx_at_Noon
0,18688431A1,94f9b4e7,b76d06,5968,826,CR0,1,10.72,7500,Late Night,0,0,1,0,0,0
1,2164986A1,648e19cf,718cc6,5499,826,DE14,81,21.19,4500,Late Night,0,0,1,0,0,0
2,31294145A1,c0ffab1b,94cafc,5735,442,NaN,81,5.04,9500,Late Night,0,0,1,0,0,0
3,11162049A1,038099dd,7d5803,5499,826,NR1,81,21.00,7500,Late Night,0,0,1,0,0,0
4,17067235A1,3130363b,12ca76,5411,826,M50,81,47.00,10500,Late Night,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118616,103590747A1,d3c036ac,0b1dae,6300,826,NN1,81,50.91,7500,Night,0,0,0,0,1,0
118617,18903650A1,e0ad403a,e5d3c5,5812,826,TW9,5,71.00,10500,Night,0,0,0,0,1,0
118618,103582443A1,d3c036ac,0b1dae,6300,826,NN1,81,17.00,8500,Night,0,0,0,0,1,0
118619,14841868A1,ec212849,1ab20c,4812,826,SL1,1,11.00,7500,Late Night,0,0,1,0,0,0


# eventID

In [267]:
# similar to transaction time 'eventId' has a very high cardinality as well with 100% unique values and it should not be used to train the model. Hence we drop this column from our feature set.

In [268]:
data_features = data_features.drop(columns=['eventId'])

# Account Number

In [269]:
# Account Number has just 766 unique values which is 0.6% of the total values. Account number as it is can't be a great predictor as it is just an id and does not provide any information. But instead 
# the frequency of the account numbers can be calculated and used as a feature. The number of transactions by the same account is definitely a great predictor for fraud detection systems.
# High number of transactions from the same account can signal towards a probable fraud.

In [271]:
acc_freq = data_features.accountNumber.value_counts()
acc_freq = pd.DataFrame(acc_freq)
acc_freq["name"] = acc_freq.index
acc_freq.rename(columns = {'accountNumber':'acc_freq', 'name':'accountNumber'}, inplace = True)
data_features = data_features.merge(acc_freq, on = 'accountNumber')
data_features = data_features.drop(columns=['accountNumber'])
data_features

,transactionTime,merchantId,mcc,merchantCountry,merchantZip,posEntryMode,transactionAmount,availableCash,isFraud,part_of_day,acc_freq
0,2017-01-01 00:00:00+00:00,b76d06,5968,826,CR0,1,10.72,7500,0.0,Late Night,362
1,2017-01-05 08:05:55+00:00,46322b,4816,826,L39LQ,81,274.92,8500,0.0,Late Night,362
2,2017-01-05 07:51:40+00:00,46322b,4816,826,L39LQ,81,11.00,8500,0.0,Late Night,362
3,2017-01-06 01:36:16+00:00,71d326,5968,826,UB11,1,8.45,7500,0.0,Late Night,362
4,2017-01-06 21:06:06+00:00,b76d06,5968,826,CR0,1,11.98,8500,0.0,Late Night,362
...,...,...,...,...,...,...,...,...,...,...,...
118616,2018-01-22 10:17:38+00:00,c647f3,6011,826,PR7,5,149.00,1500,0.0,Late Night,6
118617,2018-01-22 10:30:19+00:00,c647f3,6011,826,PR7,5,149.00,1500,0.0,Late Night,6
118618,2018-01-22 14:56:30+00:00,f513f2,5977,826,PR7,5,29.08,1500,0.0,Late Night,6
118619,2018-01-27 02:55:57+00:00,c647f3,6011,826,PR7,5,249.00,1500,0.0,Late Night,6


# Merchant Id

In [110]:
# Merchant Id is a unique id of the merchant and has 33327 unique values. It should not be used as it is while training the model because of its very high cardinality. Frequency of the merchant id could be used but there is a high chance that it gets correlated with 'mcc' as similar merchant ids will always have similar mcc.
#  Hence we drop this column from our feature set.

In [111]:
data_features = data_features.drop(columns=['merchantId'])

# MCC


In [112]:
# MCC represents the merchant category code of the merchant. It specifies the type of goods or services the merchant provides. It has a high cardinality as well and hence we use the frequency of MCC instead of the MCC codes. 
# The frequency will represent the number of times a specific type of service or goods category appeared. 

In [113]:
mcc_freq = data_features.mcc.value_counts()
mcc_freq = pd.DataFrame(mcc_freq)
mcc_freq["mcc_code"] = mcc_freq.index
mcc_freq.rename(columns = {'mcc':'mcc_freq', 'mcc_code':'mcc'}, inplace = True)
data_features = data_features.merge(mcc_freq, on = 'mcc')

# Merchant Country

In [114]:
# It is the country of the merchant who charged for the transaction. It contains 82 different countries. Using label encoder 

In [346]:
merchant_country_freq = pd.DataFrame(data_features.merchantCountry.value_counts())
merchant_country_freq["mc"] = merchant_country_freq.index
country_list = merchant_country_freq.loc[merchant_country_freq.merchantCountry>100,"mc"]
data_features.loc[data_features["merchantCountry"].isin(country_list)==False,"merchantCountry"]="low_freq_countires"

In [347]:
data_features

,eventId,accountNumber,merchantId,mcc,merchantCountry,merchantZip,posEntryMode,transactionAmount,availableCash,part_of_day,tx_at_Early Morning,tx_at_Eve,tx_at_Late Night,tx_at_Morning,tx_at_Night,tx_at_Noon
0,18688431A1,94f9b4e7,b76d06,5968,826,CR0,1,10.72,7500,Late Night,0,0,1,0,0,0
1,2164986A1,648e19cf,718cc6,5499,826,DE14,81,21.19,4500,Late Night,0,0,1,0,0,0
2,31294145A1,c0ffab1b,94cafc,5735,442,NaN,81,5.04,9500,Late Night,0,0,1,0,0,0
3,11162049A1,038099dd,7d5803,5499,826,NR1,81,21.00,7500,Late Night,0,0,1,0,0,0
4,17067235A1,3130363b,12ca76,5411,826,M50,81,47.00,10500,Late Night,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118616,103590747A1,d3c036ac,0b1dae,6300,826,NN1,81,50.91,7500,Night,0,0,0,0,1,0
118617,18903650A1,e0ad403a,e5d3c5,5812,826,TW9,5,71.00,10500,Night,0,0,0,0,1,0
118618,103582443A1,d3c036ac,0b1dae,6300,826,NN1,81,17.00,8500,Night,0,0,0,0,1,0
118619,14841868A1,ec212849,1ab20c,4812,826,SL1,1,11.00,7500,Late Night,0,0,1,0,0,0


In [249]:
pd.get_dummies(data_features['merchantCountry'])

,124,196,208,250,276,292,372,380,442,528,724,756,826,840,Others
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118616,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
118617,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
118618,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
118619,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [240]:
data_features

,transactionTime,eventId,accountNumber,merchantId,mcc,merchantCountry,merchantZip,posEntryMode,transactionAmount,availableCash,merchantCountry_freq
0,2017-01-01T00:00:00Z,18688431A1,94f9b4e7,b76d06,5968,826,CR0,1,10.72,7500,95616
1,2017-01-01T00:49:03Z,2164986A1,648e19cf,718cc6,5499,826,DE14,81,21.19,4500,95616
2,2017-01-01T00:15:07Z,11162049A1,038099dd,7d5803,5499,826,NR1,81,21.00,7500,95616
3,2017-01-01T00:37:09Z,17067235A1,3130363b,12ca76,5411,826,M50,81,47.00,10500,95616
4,2017-01-01T01:39:19Z,26312005A1,d5a8f4bc,bb3ef2,5968,826,KT11,1,312.72,10500,95616
...,...,...,...,...,...,...,...,...,...,...,...
118616,2017-11-28T08:18:26Z,93946923A1,a09baa2f,f5bc1d,5712,312,NaN,5,107.54,2500,2
118617,2017-12-09T23:47:53Z,15277454A1,d4e43604,dcfa8f,5399,566,NaN,81,364.86,7500,1
118618,2017-12-24T20:54:34Z,89825209A1,3158eaf3,9b324c,5691,410,NaN,1,37.61,6500,1
118619,2018-01-14T14:59:29Z,81464676A1,50515892,e6050e,5311,158,NaN,5,32.69,1500,2


# Merchant Zip

In [122]:
# It is the zip code of the postal address of the merchant. This column cointains 23005 missing values and the best method is to drop the column.
print('The number of na values for the feature "merchantZip" is', data_features.apply(lambda x[i] : x[i].isnull().sum()).to_dict().get('merchantZip'))

The number of na values for the feature "merchantZip" is None


In [116]:
data_features = data_features.drop(columns=['merchantZip'])

# POS Entry Mode

In [117]:
# It represents the point of sale entry mode and there are just 10 distinct values hence we should use a one hot encoder. This can be done using the 
# get_dummies function from pandas.

In [118]:
pos_dummies = pd.get_dummies(data_features['posEntryMode'],prefix='pos_mode')
data_features = pd.concat([data_features,pos_dummies],axis=1)
data_features = data_features.drop(columns=['posEntryMode'])
data_features

,mcc,merchantCountry,transactionAmount,availableCash,isFraud,acc_freq,mcc_freq,pos_mode_0,pos_mode_1,pos_mode_2,pos_mode_5,pos_mode_7,pos_mode_79,pos_mode_80,pos_mode_81,pos_mode_90,pos_mode_91
0,5968,826,10.72,7500,0.0,362,2219,0,1,0,0,0,0,0,0,0,0
1,5968,826,8.45,7500,0.0,362,2219,0,1,0,0,0,0,0,0,0,0
2,5968,826,11.98,8500,0.0,362,2219,0,1,0,0,0,0,0,0,0,0
3,5968,826,25.22,7500,0.0,362,2219,0,1,0,0,0,0,0,0,0,0
4,5968,826,8.38,7500,0.0,362,2219,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118616,3066,840,122.91,500,0.0,243,2,0,1,0,0,0,0,0,0,0,0
118617,3066,840,309.67,1500,0.0,243,2,0,1,0,0,0,0,0,0,0,0
118618,3602,840,38.26,1500,0.0,243,1,0,0,0,0,0,0,0,0,1,0
118619,3034,826,1230.04,8500,0.0,37,1,0,0,0,0,0,0,0,1,0,0


# Transaction Amount

In [191]:
# This column contains a certain number of negative values which are not veyr high in magnitude. The minimum tranasction amount is "-0.15". 
# We can either reomove these 183 values or make them zero. It wont make a huge difference if I perform any of these two methods
# but leaving the negative values as it is can degrade the performance of our classifier. I chose to remove the observations as -ve transaction values don't make sense
# and making them zero without any strong reason would not make any sense. As far as the skewness of this feature is concerned, I do not plan to use a parametric model 
# hence transforming the values according to a specific distribution won't make sense. 

In [203]:
neg_indexes = data_features[ data_features['transactionAmount'] < 0 ].index
# Delete these row indexes from dataFrame
data_features.drop(neg_indexes , inplace=True)

# Available Cash

In [123]:
# This column seems to be perfectly fine and can be used in our prediction models as it is. 

# Data set balance